In [8]:
from data import *
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
df = read_data()

In [5]:
corr_matrix = df.corr()
corr_matrix

,midblock_sig,lat,lon,signalized_ind,approaches,on_system,dist_near_school_mi,dist_near_hops_mi,transit_stops_025mi_count,sidewalk_lenght_150ft_ft,aadt_lane_major,aadt_lane_minor,a_rural,a_small_urban,a_urbanized,a_large_urban,DVMT_major,lanes_major,lane_width_ft_major,median_width_ft_major,shoulder_width_ft_major,truck_perc_major,f_local_major,f_collector_major,f_arterial_major,f_unknown_major,d_1way_major,d_2way_undiv_major,d_2way_divid_major,DVMT_minor,lanes_minor,lane_width_ft_minor,median_width_ft_minor,truck_perc_minor,f_local_minor,shoulder_width_ft_minor,f_collector_minor,f_arterial_minor,f_unknown_minor,d_1way_minor,d_2way_undiv_minor,d_2way_divid_minor,speed_lim_mph_major,speed_lim_mph_minor,tot_WMT,tot_WMT_pop,tot_WMT_sqmi,tot_crash_count,crash
midblock_sig,1.000000,-0.017541,0.015115,0.360568,0.051080,0.017646,-0.019745,-0.031549,0.053847,0.027144,0.108314,0.216496,-0.032717,-0.016709,-0.014365,0.048902,0.098831,0.132425,0.021053,0.017829,0.009467,0.001607,-0.075243,-0.005384,0.099020,0.031309,0.023663,-0.122899,0.125214,0.226130,0.218104,0.052900,0.015438,0.011971,-0.146135,0.012234,0.054403,0.202242,0.007341,0.044518,-0.151166,0.165368,-0.066658,-0.120242,0.027166,0.070652,0.059507,0.227812,0.070179
lat,-0.017541,1.000000,-0.214962,-0.019737,0.045596,0.058673,0.111874,0.070186,-0.020292,-0.037172,-0.048871,-0.058400,0.168767,0.014835,0.071269,-0.203888,-0.018966,0.009819,-0.021882,0.003130,0.028500,0.088474,-0.015260,0.024495,-0.005739,-0.001889,-0.012262,0.001184,0.005103,-0.016088,0.000796,-0.066655,0.004488,0.040090,0.000031,0.014577,0.001797,-0.003386,0.000070,-0.002747,0.008034,-0.008428,0.050421,-0.005308,-0.147025,-0.041450,-0.096229,-0.039162,-0.067167
lon,0.015115,-0.214962,1.000000,0.017366,-0.094840,-0.006283,-0.051075,-0.042354,-0.017892,-0.030306,0.031597,0.031976,-0.070873,-0.047498,0.020284,0.081462,0.033251,0.004095,-0.006824,-0.001018,-0.013187,-0.076469,-0.003525,0.002252,0.002630,-0.001578,0.000489,-0.041267,0.045954,0.017129,0.019184,0.044637,0.001301,-0.037997,-0.003113,-0.001669,-0.000289,0.007102,-0.000375,-0.001960,-0.022553,0.032827,-0.030916,-0.012589,0.169018,0.099312,0.041136,0.009711,-0.012045
signalized_ind,0.360568,-0.019737,0.017366,1.000000,0.134356,0.095303,-0.056838,-0.079660,0.125558,0.157442,0.221502,0.446454,-0.079374,-0.015171,0.000055,0.080437,0.179186,0.320394,0.093757,0.044992,0.057569,0.018230,-0.206534,-0.019091,0.289259,0.030747,0.089902,-0.203189,0.180391,0.391062,0.388493,0.203391,0.052091,0.054150,-0.385454,0.093459,0.230094,0.371946,0.028411,0.163955,-0.280880,0.230247,-0.223951,-0.274989,0.022450,0.110719,0.092991,0.512081,0.222471
approaches,0.051080,0.045596,-0.094840,0.134356,1.000000,0.170800,-0.028788,0.013252,0.042959,0.119855,0.244273,0.149105,0.050016,0.077857,0.057435,-0.130915,0.125521,0.152003,0.149456,0.111920,0.164566,0.122003,-0.247283,0.103382,0.174196,0.189283,0.076213,-0.175435,0.156490,0.113271,0.089358,0.080854,0.029154,0.054738,-0.174615,0.082477,0.126192,0.123003,0.033628,0.086565,-0.108787,0.067310,-0.134139,-0.118260,-0.158738,-0.006227,0.031423,0.148067,0.256191
on_system,0.017646,0.058673,-0.006283,0.095303,0.170800,1.000000,0.260502,0.260526,-0.029728,0.377287,0.308276,0.137240,0.322176,0.030649,-0.030269,-0.287672,0.103264,0.181723,0.301859,0.123629,0.709942,0.647850,-0.603600,0.327910,0.408890,0.178902,0.138369,-0.120599,0.062496,0.069174,0.037604,-0.056945,0.045915,0.217053,-0.253933,0.213644,0.209526,0.126138,0.068798,0.146831,-0.129400,0.038474,-0.147670,-0.125690,-0.159055,-0.179217,-0.218495,0.121334,0.140679
dist_near_school_mi,-0.019745,0.111874,-0.051075,-0.056838,-0.028788,0.260502,1.000000,0.532125,-0.049353,-0.064773,-0.086500,-0.098024,0.576066,-0.104575,-0.112522,-0.374071,-0.043654,-0.116667,-0.152246,-0.001483,0.136105,0.269105,-0.068734,0.130508,-0.046576,-0.013453,-0.036122,0.071180,-0.060767,-0.034527,-0.040861,-0.345741,-0.002086,0.121552,-0.020660,0.046117,0.039567,-0.030431,0.002644,-0.015978,0.028565,-0.024076,0.21

In [14]:
#sn.heatmap(corr_matrix, annot=True)
#plt.show()
corr_matrix.to_csv("data/corr_matrix.csv")

In [12]:
corr_matrix > 0.5

,midblock_sig,lat,lon,signalized_ind,approaches,on_system,dist_near_school_mi,dist_near_hops_mi,transit_stops_025mi_count,sidewalk_lenght_150ft_ft,aadt_lane_major,aadt_lane_minor,a_rural,a_small_urban,a_urbanized,a_large_urban,DVMT_major,lanes_major,lane_width_ft_major,median_width_ft_major,shoulder_width_ft_major,truck_perc_major,f_local_major,f_collector_major,f_arterial_major,f_unknown_major,d_1way_major,d_2way_undiv_major,d_2way_divid_major,DVMT_minor,lanes_minor,lane_width_ft_minor,median_width_ft_minor,truck_perc_minor,f_local_minor,shoulder_width_ft_minor,f_collector_minor,f_arterial_minor,f_unknown_minor,d_1way_minor,d_2way_undiv_minor,d_2way_divid_minor,speed_lim_mph_major,speed_lim_mph_minor,tot_WMT,tot_WMT_pop,tot_WMT_sqmi,tot_crash_count,crash
midblock_sig,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
lat,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
lon,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
signalized_ind,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
approaches,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
on_system,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
dist_near_school_mi,False,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
dist_near_hops_mi,False,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
transit_stops_025mi_count,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
sidewalk_lenght_150ft_ft,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
